In [194]:
import skimage
import matplotlib.pyplot as plt
import numpy as np
import pywt
import scipy
import os

from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
def get_noise_image(image):
    ret = np.zeros(image.shape, dtype='uint8')
    ret[:,:,0] = skimage.filters.rank.median(image[:,:,0], np.ones((5,5)))
    ret[:,:,1] = skimage.filters.rank.median(image[:,:,1], np.ones((5,5)))
    ret[:,:,2] = skimage.filters.rank.median(image[:,:,2], np.ones((5,5)))

    return image.astype('float64')/255.0 - ret.astype('float64')/255.0

In [ ]:
def extract_features(image):
    subimages = []
    subimages.append(get_noise_image(image))

    coeffs = pywt.dwt2(noise, 'db2')

    subimages.append(coeffs[0])
    subimages.append(coeffs[1][0])
    subimages.append(coeffs[1][1])
    subimages.append(coeffs[1][2])

    line = []
    for im in subimages:
        for c in range(3):
            nobs, minmax, mean, variance, skewness, kurtosis = scipy.stats.describe(im[:,:,c].ravel())    
            line = line + [minmax[0],minmax[1],mean,variance,skewness,kurtosis]
    return line